In [1]:
from contextlib import closing
import json
import pandas as pd
import re
import requests
import snowflake.connector
import time
from nltk.corpus import stopwords
import string
from nltk.tokenize import wordpunct_tokenize as tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
schema = 'REPORTING'

In [4]:
sql = """
        SELECT D.CHANNEL_ID, D.VIDEO_ID, D.SUITABILITY_CATEGORY, D.RISK_TIER, S.TAXONOMY_NAME, 
        S.SCORE, S.DECISION
        FROM prod.analyst.DV_ANALYSIS_3 D
        JOIN prod.REPORTING.LATEST_SCORES AS S 
            ON D.VIDEO_ID = S.CONTENT_ID    
      """

In [5]:
def connection(schema):
    conn = snowflake.connector.connect(
        host="localhost",
        port="10015",
        user="sdm",
        password="sdm",
        account="dummy",
        protocol="http",
        warehouse="ENGINEER_WH",
        database="PROD",
        schema=schema,
    )
    return conn

def get_results(sql):
    conn = connection(schema)
    with closing(conn.cursor()) as cursor:
        results = conn.cursor().execute(sql)
        
    # Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
    df = results.fetch_pandas_all()
    return df

In [6]:
df = get_results(sql)

In [7]:
df.describe()

,CHANNEL_ID,VIDEO_ID,SUITABILITY_CATEGORY,RISK_TIER,TAXONOMY_NAME,SCORE,DECISION
count,77348880,77348880,77348880,77348880,77348880,77348880,47672636
unique,69788,340449,34,4,260,11889404,2
top,UCUk7VggtJdo9XYTy3Z5QVAw,ubEwaITzTKc,Hobbies & Interests: Video & Computer Games,N/A,Fitness - V1,0,False
freq,257906,1480,25956782,64678774,417747,18381603,41451576


In [9]:
def split_dataframe(df, chunk_size = 1000000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [10]:
chunks = split_dataframe(df, 1000000)

In [11]:
len(chunks)

78

In [12]:
chunks[0].head()

,CHANNEL_ID,VIDEO_ID,SUITABILITY_CATEGORY,RISK_TIER,TAXONOMY_NAME,SCORE,DECISION
0,UCJZCTuXWrrJflyNvOK1LCMQ,wZnhqv1nSUQ,Automotive,N/A,Green Living - V1,0.083816774,None
1,UCiytbL7aJQ_ak05CWfnOqgQ,bGYqHwIZAl0,Hobbies & Interests: Video & Computer Games,N/A,Green Living - V1,0.062662356,None
2,UCiytbL7aJQ_ak05CWfnOqgQ,bGYqHwIZAl0,Hobbies & Interests,N/A,Green Living - V1,0.062662356,None
3,UCaTznQhurW5AaiYPbhEA-KA,Ge3aKEmZcqY,Education,N/A,Rock Music - V1,0.054847192,None
4,UCauDySKxJnYuKmkMqcs5--w,F9-BIUCosmQ,Profanity,Medium Risk,Japanese Travel & Tourism - V1,0.074226685,None


In [13]:
c2 = []

for i in chunks:
    i[['CHANNEL_ID','VIDEO_ID']] = i[['CHANNEL_ID','VIDEO_ID']].astype(str)
    i['SCORE'] = i['SCORE'].astype(float)
    c2.append(i)

/Users/onb/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
<ipython-input-13-448f57c7eca4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i['SCORE'] = i['SCORE'].astype(float)


In [14]:
for i in c2:
    DV_RISK = []

    for x in i['RISK_TIER']:
        if x == 'N/A':
            DV_RISK.append('Minimal')
        else:
            DV_RISK.append(x)
    i['DV_RISK'] = DV_RISK
    i.drop('RISK_TIER', inplace=True, axis = 1)

<ipython-input-14-861952d55e7a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i['DV_RISK'] = DV_RISK
/Users/onb/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
import math
import re
from collections import Counter
import pandas as pd

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [16]:
for i in c2:    
    i['vector1']=i['SUITABILITY_CATEGORY'].apply(lambda x: text_to_vector(x)) 
    i['vector2']=i['TAXONOMY_NAME'].apply(lambda x: text_to_vector(x)) 
    i['simscore']=i.apply(lambda x: get_cosine(x['vector1'],x['vector2']),axis=1)

<ipython-input-16-59455846335a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i['vector1']=i['SUITABILITY_CATEGORY'].apply(lambda x: text_to_vector(x))
<ipython-input-16-59455846335a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i['vector2']=i['TAXONOMY_NAME'].apply(lambda x: text_to_vector(x))
<ipython-input-16-59455846335a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [17]:
rsd = pd.concat(c2)

In [18]:
rsd.describe()

,SCORE,simscore
count,7.734888e+07,7.734888e+07
mean,1.298421e-01,5.121532e-03
std,2.640623e-01,4.612644e-02
min,0.000000e+00,0.000000e+00
25%,1.297559e-02,0.000000e+00
50%,4.756835e-02,0.000000e+00
75%,7.676369e-02,0.000000e+00
max,1.000000e+00,7.071068e-01


In [21]:
#dataframe where DV has a safety category

dv_safety_flag = rsd[rsd['DV_RISK'] != 'Minimal'][:100000]
dv_safety_flag.to_pickle('DV_Safety_Flag_Risk_Data.pkl')

In [23]:
#dataframe where zefr has a safety category

zefr_safety_flag = rsd[(rsd['TAXONOMY_NAME'].str.contains("Brand Safety")) | (rsd['TAXONOMY_NAME'].str.contains("4A"))][:100000]
zefr_safety_flag.to_pickle('ZEFR_Safety_Flag_Risk_Data.pkl')

In [17]:
#dataframe where DV has a safety category, but zefr/garm do not

dv_safety_only = rsd[(rsd['DV_RISK'] != 'Minimal') & ((~rsd['TAXONOMY_NAME'].str.contains("Brand Safety")) & (~rsd['TAXONOMY_NAME'].str.contains("4A")))][:100000]
dv_safety_only.to_pickle('DV_Safety_Only_Risk_Data.pkl')

In [18]:
#dataframe where DV has a safety category and zefr/garm mention brand safety or 4A

dv_zefr_safety = rsd[(rsd['DV_RISK'] != 'Minimal') & ((rsd['TAXONOMY_NAME'].str.contains("Brand Safety")) | (rsd['TAXONOMY_NAME'].str.contains("4A")))][:100000]
dv_zefr_safety.to_pickle('DV_Zefr_Safety_Risk_Data.pkl')

In [19]:
#dataframe where DV and zefr have similar concept categories

dv_zefr_similar_concepts = rsd[rsd['SCORE']>0.5][:100000]
dv_zefr_similar_concepts.to_pickle('DV_Zefr_SimilarConcept_Risk_Data.pkl')

#dataframe where DV and zefr have disparate concept categories

dv_zefr_diff_concepts = rsd[rsd['SCORE']==0][:100000]
dv_zefr_diff_concepts.to_pickle('DV_Zefr_DiffConcept_Risk_Data.pkl')